In [186]:
import pandas as pd
import numpy as np
from numpy.linalg import inv
from itertools import cycle
%matplotlib inline
import matplotlib.pyplot as plt
plt.matplotlib.rcParams['figure.figsize'] = (6,6)

Question 1

In [187]:
def Path_simulation(n, T):
    S0 = 40
    periods = T * 50
    sigma = .2
    r = .06
    delta = 1 / 50.0
    discFactor = np.exp(-r * T)
    c = np.zeros(n);
    z = np.random.normal(0, 1, (n, periods))
    z_anti = -z
    prices_paths = np.zeros((n, periods))
    prices_paths_anti = np.zeros((n, periods))
    prices_paths[:, 0] = S0 * np.exp((r - .5 * sigma ** 2) * delta + sigma * np.sqrt(delta) * z[:, 0])
    prices_paths_anti[:, 0] = S0 * np.exp((r - .5 * sigma ** 2) * delta + sigma * np.sqrt(delta) * z_anti[:, 0])
    for i in xrange(1, periods):
        prices_paths[:, i] = prices_paths[:, i - 1] *\
                                np.exp((r - .5 * sigma ** 2) * delta +
                                       sigma * np.sqrt(delta) * z[:, i])
        prices_paths_anti[:, i] = prices_paths_anti[:, i - 1] *\
                                    np.exp((r - .5 * sigma ** 2) * delta +
                                        sigma * np.sqrt(delta) * z_anti[:, i])
    return (prices_paths, prices_paths_anti)

In [194]:
def xy_transfer(X, y):
    
    X = X.reshape(-1, 1)
    y = y.reshape(-1, 1)
    x0 = np.ones(X.shape)
    x1 = np.exp(-X / 2)
    x2 = x1 * (1 - X)
    x3 = x1 * (1 - 2 * X + X ** 2 / 2)
    new_X = np.concatenate((x0, x1, x2, x3), axis=1)
    return(np.matrix(new_X), np.matrix(y))



def Path_Regression(n, T, path, K):
    r = .06
    delta = 1 / 50.0
    periods = T * 50
    discFactor = np.exp(-r * delta)
    Tau = (np.ones(n) * periods).astype(int) - 1
    selec_v = np.arange(n)
    Payoff_M = np.where(K > path, K - path, 0)
    for i in range(periods - 2, -1, -1):
        discFactor_ = discFactor ** (Tau - i)
        y = Payoff_M[selec_v, Tau] * discFactor_
        in_the_money = Payoff_M[:, i] > 0
        X = path[:, i] / K
        y = np.where(in_the_money, y, 0)
        X, y = xy_transfer(X, y)
        X[~in_the_money, :] = 0
        y_exc = np.where(in_the_money, Payoff_M[:, i], 0)
        Y_p = X * (inv(X.T * X) * X.T * y)
        Tau[y_exc > Y_p.A1] = i 
    
    discFactor_ = discFactor ** (Tau + 1)
    all_payoff = Payoff_M[selec_v, Tau] * discFactor_
    return all_payoff



def anti_simulation(n, T, K):
    path1, path2 = Path_simulation(n, T)
    x1 = Path_Regression(n, T, path1, K)
    x2 = Path_Regression(n, T, path2, K)
    mean_price = (x1 + x2) / 2
    price = mean_price.mean()
    ste = mean_price.std() / np.sqrt(n)
    return price, ste

In [195]:
anti_simulation(50000, 1, 40)

(2.3179262015417517, 0.0053773632288531079)

In [196]:
anti_simulation(50000, 2, 40)

(2.8853133264536028, 0.0065147846997584196)

Question 2

In [309]:
from scipy.optimize import curve_fit
from scipy.stats import norm

In [292]:
K = 120

def bslink(X, theta):
    S0 = 100
    r = .05
    sigma = .2
    T = 1
    discFactor = np.exp(-r * T)
    rnm = np.exp((r - .5 * sigma ** 2) * T + sigma * np.sqrt(T) * X)
    result1 = np.where((S0 * rnm - K) > 0, (S0 * rnm - K) * discFactor, 0)
    result2 = (np.exp(-theta * X + 0.5 * theta ** 2)) ** 0.5
    return result1 * result2

In [293]:
indp = np.random.normal(0, 1, 10000)
dep = np.zeros(10000)
curve_fit(bslink, indp, dep, p0=1)

(array([ 1.66233759]), array([[ 0.00198571]]))

In [366]:
K = 140

indp = np.random.normal(0, 1, 10000)
dep = np.zeros(10000)
curve_fit(bslink, indp, dep, p0=1)

(array([ 2.20418366]), array([[ 0.00336042]]))

In [365]:
K = 160

indp = np.random.normal(0, 1, 10000)
dep = np.zeros(10000)
curve_fit(bslink, indp, dep, p0=1)

(array([ 2.73525429]), array([[ 0.00556489]]))

In [335]:
def adjustment_factor(theta, Z):

    adj_factor = np.exp(-theta * Z + 0.5 * theta ** 2)
    return adj_factor
    

def E_option_pricing_importance(n, K, theta):

    r = .05
    sigma = .2
    S0 = 100
    T = 1
    delta = T
    discFactor = np.exp(-r * T)
    c = np.zeros(n);
    z = np.random.normal(0, 1, n)
    z_c = z + theta
    ST = S0 * np.exp((r - .5 * sigma ** 2) * delta + sigma * np.sqrt(delta) * z_c)
    adj = adjustment_factor(theta, z_c)
    pay_off = ST - K
    payoff = np.where(pay_off > 0, pay_off, 0)
    c = payoff * discFactor * adj
    return (c.mean(), c.std() / np.sqrt(n))

In [336]:
E_option_pricing_importance(10000, 120, 1.66233759)

(3.2702729314333703, 0.021666095099231768)

In [339]:
E_option_pricing_importance(10000, 140, 2.2268528)

(0.78144794771049741, 0.006686033896089242)

In [340]:
E_option_pricing_importance(10000, 160, 2.75997713)

(0.15923325172643168, 0.0015909132154081926)

In [341]:
def bslink2(X, m, s):
#     m, s = theta
    S0 = 100
    r = .05
    sigma = .2
    T = 1
    discFactor = np.exp(-r * T)
    rnm = np.exp((r - .5 * sigma ** 2) * T + sigma * np.sqrt(T) * X)
    result1 = np.where((S0 * rnm - K) > 0, (S0 * rnm - K) * discFactor, 0)
    result2 = np.exp(-.5 * (X ** 2 - ((X - m) / np.exp(s)) ** 2)) * np.exp(s)
    return result1 * result2

In [400]:
K = 120
indp = np.random.normal(0, 1, 10000)
dep = np.zeros(10000)
x = curve_fit(bslink2, indp, dep, p0=(1, 1))
m, exps = x[0][0], x[0][1]
print m, np.exp(exps)

1.56282312802 0.65603514734


In [371]:
K = 140
indp = np.random.normal(0, 1, 10000)
dep = np.zeros(10000)
x = curve_fit(bslink2, indp, dep, p0=(1, 1))
m, exps = x[0][0], x[0][1]
print m, np.exp(exps)

2.15093832242 0.529687635453


In [397]:
K = 160
indp = np.random.normal(0, 1, 10000)
dep = np.zeros(10000)
x = curve_fit(bslink2, indp, dep, p0=(1, 1))
m, exps = x[0][0], x[0][1]
print m, np.exp(exps)

2.70122589344 0.474950476118


In [382]:
def adjustment_factor_ms(m, s, Z):

    adj_factor = np.exp(-.5 * (Z ** 2 - ((Z - m) / s) ** 2)) * s
    return adj_factor
    

def E_option_pricing_importance_ms(n, K, theta, s):

    r = .05
    sigma = .2
    S0 = 100
    T = 1
    delta = T
    discFactor = np.exp(-r * T)
    c = np.zeros(n);
#     z = np.random.normal(0, 1, n)
#     z_c = z + theta
    z_c = np.random.normal(theta, s, n)
    ST = S0 * np.exp((r - .5 * sigma ** 2) * delta + sigma * np.sqrt(delta) * z_c)
    adj = adjustment_factor_ms(theta, s, z_c)
    pay_off = ST - K
    payoff = np.where(pay_off > 0, pay_off, 0)
    c = payoff * discFactor * adj
    return (c.mean(), c.std() / np.sqrt(n))

In [383]:
E_option_pricing_importance_ms(10000, 120, 1.56993981901, 0.614311825363)

(3.2350703579755571, 0.01398934223833598)

In [390]:
E_option_pricing_importance_ms(10000, 140, 2.15093832242, 0.529687635453)

(0.7810180409596863, 0.0039222227422787631)

In [401]:
E_option_pricing_importance_ms(10000, 160, 2.70122589344, 0.474950476118)

(0.15821282634766859, 0.00085400969488653815)